# Bird Observation Data

### Import libraries and configure environment variable

In [1]:
import requests
import pandas as pd
import os

from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get('EBIRD_API_KEY')

### GET Requests to eBird API

In [2]:
# Fetches all recent recorded observations in PA
recent_observations_url = 'https://api.ebird.org/v2/data/obs/US-PA/recent'
recent_observations_params = {'key': os.environ.get('EBIRD_API_KEY')}
recent_observations = requests.get(recent_observations_url, params=recent_observations_params)

# Handle potential JSONDecodeError
try:
    recent_observations_data = recent_observations.json()
except requests.exceptions.JSONDecodeError:
    print("Error decoding JSON response for recent_observations:")
    print(recent_observations.text)
    exit(1)

### Create DataFrames

In [3]:
# Create a DataFrame to hold API response data
pa_observations_df = pd.DataFrame(recent_observations_data)

### Transform
- Drop unnecessary columns
- Rename columns for clarity
- Remove duplicate data

In [4]:
# Drop unneeded columns from observation data and rename remaining for clarity

# Specify the columns to be dropped
columns_to_drop = ['obsValid', 'obsReviewed', 'exoticCategory']
# Drop the specified columns from the DataFrame
pa_observations_df = pa_observations_df.drop(columns=columns_to_drop)\
                                        .rename(columns={'speciesCode': 'species_code', 'comName': 'common_name', 'sciName': 'scientific_name', 
                                                        'locId': 'location_id', 'locName': 'location_name', 'obsDt' : 'observation_date', 
                                                        'howMany': 'quantity_observed', 'lat': 'latitude', 'lng': 'longitude', 
                                                        'locationPrivate': 'location_private', 'subId': 'submission_id'})

In [5]:
# Remove duplicate entries from DataFrame
pa_observations_deduplicated_df = pa_observations_df.drop_duplicates(keep='first')

# Transform NaN values in the 'Quantity_Observed' column to a default value of 1
pa_observations_deduplicated_df['quantity_observed'].fillna(1, inplace=True)

In [6]:
print(pa_observations_deduplicated_df.head())

  species_code             common_name         scientific_name location_id   
0       indbun          Indigo Bunting        Passerina cyanea   L17711128  \
1       norpar         Northern Parula     Setophaga americana   L17711128   
2       chswar  Chestnut-sided Warbler  Setophaga pensylvanica   L17711128   
3       swathr       Swainson's Thrush      Catharus ustulatus   L17711128   
4      ovenbi1                Ovenbird     Seiurus aurocapilla   L17711128   

                            location_name  observation_date   
0  Greycar Rion Memorial Refuge (my yard)  2023-09-27 00:00  \
1  Greycar Rion Memorial Refuge (my yard)  2023-09-27 00:00   
2  Greycar Rion Memorial Refuge (my yard)  2023-09-27 00:00   
3  Greycar Rion Memorial Refuge (my yard)  2023-09-27 00:00   
4  Greycar Rion Memorial Refuge (my yard)  2023-09-27 00:00   

   quantity_observed   latitude  longitude  location_private submission_id  
0                1.0  39.848812 -75.518015              True    S150842201 

### Write/append data to CSV files

In [7]:
# Append de-duplicated recent observations data to a CSV file
import csv
# pa_observations_deduplicated_df.to_csv('recent_observations.csv', index=False)
pa_observations_deduplicated_df.to_csv('recent_observations.csv', mode='a', header=True, index=False)

### Write observation data from Pandas DataFrame to SQLite database

#### Create SQLite Database

In [8]:
import sqlite3

# Specify column names
column_names = ['species_code', 'common_name', 'scientific_name', 'location_id', 'location_name',
                'observation_date', 'quantity_observed', 'latitude', 'longitude', 'location_private', 'submission_id']

connection = sqlite3.connect("pa-observation-data.db")
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS Observations;")

# Create the table with the appropriate columns
create_table_query = "CREATE TABLE Observations ({})".format(", ".join(column_names))
cursor.execute(create_table_query)

#### Write data to database

In [9]:
# Insert values from the DataFrame into the table
for i in range(len(pa_observations_deduplicated_df)):
    cursor.execute("INSERT INTO Observations VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", pa_observations_deduplicated_df.iloc[i])

connection.commit()
connection.close()

In [10]:
# home = os.environ.get('LOCATION')
# pa_observations_df[(pa_observations_df.Location_ID == home)]

# Confirm most recent recorded observation of Pileated Woodpecker
pa_observations_deduplicated_df[(pa_observations_deduplicated_df.common_name == 'Pileated Woodpecker')]

,species_code,common_name,scientific_name,location_id,location_name,observation_date,quantity_observed,latitude,longitude,location_private,submission_id
78,pilwoo,Pileated Woodpecker,Dryocopus pileatus,L7915631,Rothrock SF--West Chestnut St.,2023-09-26 18:10,1.0,40.729845,-77.887658,False,S150829509


In [11]:
pa_observations_deduplicated_df.dtypes

species_code          object
common_name           object
scientific_name       object
location_id           object
location_name         object
observation_date      object
quantity_observed    float64
latitude             float64
longitude            float64
location_private        bool
submission_id         object
dtype: object

### Write observation data from CSV file to PostgreSQL database

In [12]:
from sqlalchemy import create_engine

# PostgreSQL database connection details from .env file
host = os.getenv('PG_HOST')
database = os.getenv('PG_DATABASE')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}/{database}', pool_pre_ping=True)

# Type conversion of Location_ID column to remove preceding L, if necessary, to store the number as an integer.
pa_observations_deduplicated_df['location_id'] = pa_observations_deduplicated_df['location_id'].astype(str).apply(lambda x: int(x[1:]) if x.startswith('L') else int(x))

table_name = "recent_observations"
try:
    pa_observations_deduplicated_df.to_sql(table_name, engine, if_exists='append', index=False)
    print("Data insertion successful!")
except Exception as e:
    print("An error occurred during data insertion:", str(e))

Data insertion successful!
